In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

%matplotlib inline
warnings.filterwarnings('ignore')

## 목표는 0.53!!!

In [ ]:
train = pd.read_json('data/train.json')

In [ ]:
test = pd.read_json('data/test.json')

In [ ]:
listing_image_time = pd.read_csv('data/listing_image_time.csv')

In [ ]:
listing_image_time = listing_image_time.rename(columns={'Listing_Id':'listing_id'})

In [ ]:
listing_image_time.head()

In [ ]:
from datetime import datetime

In [ ]:
datetime.fromtimestamp(listing_image_time['time_stamp'][0]).day

In [ ]:
listing_image_time['time_stamp'].apply(lambda x: datetime.fromtimestamp(x).month).value_counts()

In [ ]:
listing_image_time['ts_day'] = listing_image_time['time_stamp'].apply(lambda x: datetime.fromtimestamp(x).day)
listing_image_time['ts_hour'] = listing_image_time['time_stamp'].apply(lambda x: datetime.fromtimestamp(x).hour)
listing_image_time['ts_minute'] = listing_image_time['time_stamp'].apply(lambda x: datetime.fromtimestamp(x).minute)
listing_image_time['ts_second'] = listing_image_time['time_stamp'].apply(lambda x: datetime.fromtimestamp(x).second)

In [ ]:
listing_image_time.head()

In [ ]:
# pd.to_datetime(listing_image_time['time_stamp']).dt.second.value_counts()

In [ ]:
# pd.to_datetime(listing_image_time['time_stamp']).dt.minute
# pd.to_datetime(listing_image_time['time_stamp']).dt.second
# pd.to_datetime(listing_image_time['time_stamp']).dt.year
# pd.to_datetime(listing_image_time['time_stamp']).dt.month
# pd.to_datetime(listing_image_time['time_stamp']).dt.day

In [ ]:
train.head()

In [ ]:
test.head(30)

In [ ]:
num_features = train.select_dtypes(exclude='object').columns

In [ ]:
train[num_features]

In [ ]:
test[num_features]

## Merge two dataframes

In [ ]:
all_data = pd.concat([train, test])

In [ ]:
all_data = pd.merge(all_data, listing_image_time, on='listing_id', how='left')

## EDA

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mlp
import seaborn as sns
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
plt.figure(figsize=(12, 9))
sns.distplot(all_data['bathrooms'])

In [ ]:
all_data['bathrooms'].value_counts()

In [ ]:
all_data.loc[all_data['bathrooms']>6, 'bathrooms'] = 6.5

In [ ]:
plt.figure(figsize=(12, 9))
sns.distplot(all_data['bedrooms'])

## price

In [ ]:
import numpy as np

In [ ]:
all_data['price_log'] = np.log1p(all_data['price'])

In [ ]:
right = all_data.groupby('manager_id')[['price', 'price_log', 'bedrooms', 'bathrooms', 'time_stamp', 'listing_id']].agg(['mean', 'max', 'min', 'median', 'std', 'count']).reset_index()

In [ ]:
right

In [ ]:
all_data = pd.merge(all_data, right, on='manager_id', how='left')

In [ ]:
all_data.head()

In [ ]:
# right = all_data.groupby('manager_id')['price'].mean().reset_index().rename(columns={'price':'manager_price'})

In [ ]:
# right

In [ ]:
# all_data = pd.merge(all_data, right, on='manager_id', how='left')

In [ ]:
all_data["price_t"] =all_data["price"] / all_data["bedrooms"]
all_data["room_sum"] = all_data["bedrooms"] + all_data["bathrooms"] 
all_data['price_per_room'] = all_data['price'] / all_data['room_sum']

## removing building_id less than 3

In [ ]:
df = pd.DataFrame(all_data['building_id'].value_counts() <= 3)

In [ ]:
df_list = df.loc[df['building_id'] == False].index

In [ ]:
all_data['building_id'] = all_data['building_id'].apply(lambda x: x if x in df_list else 'ETC')

In [ ]:
all_data['building_id'] = all_data['building_id'].astype('category').cat.codes

In [ ]:
all_data['building_id'].value_counts()

### Manager_id

In [ ]:
all_data['manager_id'] = all_data['manager_id'].astype('category').cat.codes

## Latitude & Longitude

In [ ]:
# from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
# scaler = StandardScaler()

In [ ]:
# all_data[['latitude', 'longitude']] = scaler.fit_transform(all_data[['latitude', 'longitude']])

In [ ]:
all_data['lat-long'] = all_data['latitude'] - all_data['longitude']
all_data['lat+long'] = all_data['latitude'] + all_data['longitude']

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
all_data.describe()

In [ ]:
all_data["XY30_1"] = all_data["latitude"] * np.cos(np.pi / 6) + all_data["longitude"] * np.sin(np.pi / 6)
all_data["XY30_2"] = all_data["longitude"] * np.cos(np.pi / 6) - all_data["latitude"] * np.sin(np.pi / 6)
all_data["XY60_1"] = all_data["latitude"] * np.cos(np.pi / 3) + all_data["longitude"] * np.sin(np.pi / 3)
all_data["XY60_2"] = all_data["longitude"] * np.cos(np.pi / 3) - all_data["latitude"] * np.sin(np.pi / 3)

## Address

In [ ]:
all_data['display_address']

In [ ]:
import re

def clean_address(address):
    address = address.lower()
    
    address= address.replace('.', '')
    
    address= address.replace('west', 'w')
    address= address.replace('east', 'e')
    address= address.replace('south', 's')
    address= address.replace('north', 'n')
    
    address = re.sub(r"(\d+)(st|nd|rd|th)\b", r"\1", address)
    
    address= address.replace('street', 'st')
    
    return address

In [ ]:
all_data['display_address'] = all_data['display_address'].apply(clean_address)

In [ ]:
all_data['street_address'] = all_data['street_address'].apply(clean_address)

### Display Address & Street Address One HOT

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()

In [ ]:
all_data['display_address'] = le.fit_transform(all_data['display_address'])

In [ ]:
all_data['street_address'] = le.fit_transform(all_data['street_address'])

## Created

In [ ]:
all_data['created'] = pd.to_datetime(all_data['created'])

In [ ]:
all_data['month'] = all_data['created'].dt.month

In [ ]:
all_data['day'] = all_data['created'].dt.day

In [ ]:
all_data['hour'] = all_data['created'].dt.hour

In [ ]:
all_data['minute'] = all_data['created'].dt.minute

In [ ]:
all_data['second'] = all_data['created'].dt.second

### Photos

In [ ]:
all_data['photo_len'] = all_data['photos'].apply(lambda x: len(x))

### Features

In [ ]:
ft = all_data["features"].apply(lambda x: " ".join(["_".join(i.split(" ")) for i in x]))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv = CountVectorizer(max_features=220, stop_words='english')

In [ ]:
vectorized = cv.fit_transform(ft)

In [ ]:
vectorized.toarray().shape

In [ ]:
vectorized.shape

In [ ]:
all_data = pd.concat([all_data, pd.DataFrame(vectorized.toarray())], axis=1)

In [ ]:
# all_data = pd.concat([all_data, pd.DataFrame(vectorized.toarray())], axis=1)

In [ ]:
# from sklearn.decomposition import PCA

In [ ]:
# pca = PCA(n_components=10)

In [ ]:
# result = pca.fit_transform(vectorized.toarray())

In [ ]:
# all_data = pd.concat([all_data, pd.DataFrame(result, columns=np.arange(1000, 1010))], axis=1)

In [ ]:
# all_data.columns

In [ ]:
all_data['features_len'] = all_data['features'].apply(len)

### Description

In [ ]:
all_data['description_len'] = all_data['description'].apply(lambda x: len(x.split()))

## Count

In [ ]:
right = all_data.groupby('manager_id')['building_id'].agg(['nunique']).reset_index()

In [ ]:
all_data = pd.merge(all_data, right, on='manager_id', how='left')

## Define Features

In [ ]:
num_features = all_data.select_dtypes(exclude='object').columns

In [ ]:
exclude_feature = [
    'created'
]

In [ ]:
num_features = [x for x in num_features if x not in exclude_feature]

In [ ]:
all_data[num_features].head()

## Define Data & Target

In [ ]:
data = all_data[:len(train)]

In [ ]:
target = all_data[len(train):]

## Define x_train, y_train, x_test

In [ ]:
x_train = data[num_features]

In [ ]:
y_train = train['interest_level']

In [ ]:
x_test = target[num_features]

## Modeling

In [ ]:
from xgboost import XGBClassifier

In [ ]:
model = XGBClassifier(n_estimators=1500, 
                      learning_rate=0.025, 
                      max_depth=6,
                      colsample_bytree=0.8, 
                      colsample_bylevel=0.9, 
                      subsample=0.8, 
                      nthread=32,
                      random_state=34, 
#                       tree_method='gpu_hist',
                     )

In [ ]:
model.fit(x_train, y_train)

In [ ]:
fi = pd.DataFrame(list(zip(num_features, model.feature_importances_))).sort_values(1, ascending=False)
fi

In [ ]:
fi.head(50)

In [ ]:
pred = model.predict_proba(x_test)

In [ ]:
pred[:5]

In [ ]:
model.classes_

In [ ]:
result = pd.DataFrame(pred, columns=model.classes_)

In [ ]:
submission = pd.read_csv('data/sample_submission.csv')

In [ ]:
submission

In [ ]:
result['listing_id'] = submission['listing_id']

In [ ]:
import datetime

In [ ]:
timestring = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

In [ ]:
filename = "submission/two-sigma-{}.csv".format(timestring)

In [ ]:
filename

In [ ]:
result.to_csv(filename, index=False)